In [ ]:
!pip install --upgrade diffusers[torch]
!pip install transformers
!pip install matplotlib
!pip install opencv-python
!pip install -U peft
!wget "https://raw.githubusercontent.com/huggingface/diffusers/main/examples/community/lpw_stable_diffusion_xl.py"

In [ ]:
from diffusers import StableDiffusionXLPipeline
from accelerate import Accelerator
import torch 
from diffusers import AutoPipelineForText2Image
import matplotlib.pyplot as plt

# Initialize the accelerator
accelerator = Accelerator()

# pipe = StableDiffusionXLPipeline.from_single_file("ultrium20V70SDXLVAE.ua2r.safetensors", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
model_id = "GraydientPlatformAPI/ultrium7"

# pipe = StableDiffusionXLPipeline.from_pretrained(model_id, torch_dtype=torch.float16, use_safetensors=True, cache_dir = "models")
# pipe.to("cuda")

pipe = StableDiffusionXLPipeline.from_single_file("./juggernautX.lIDA.safetensors", cache_dir="models")
pipe.to("cuda")

# pipe = StableDiffusionXLPipeline.from_single_file("./juggernautX.lIDA.safetensors", scheduler=scheduler,cache_dir="models")
# pipe.to("cuda")

local_lora_model_paths = ["./xpenis263.XgFC.safetensors", "./gp.safetensors"]  # Replace with the correct local path to the Lora model
lora_weights = [5, 1]

local_lora_model_paths = ["./gp.safetensors"]  # Replace with the correct local path to the Lora model
lora_weights = [0.02]

local_lora_model_paths = ["./gp.safetensors", "./ssf.safetensors"]  # Replace with the correct local path to the Lora model
lora_weights = [0.1, 1]

for lora_model_path, weight in zip(local_lora_model_paths, lora_weights):
    pipe.load_lora_weights(lora_model_path, weight=weight) 

# Load text-to-image pipe
# pipe = AutoPipelineForText2Image.from_pretrained(
#     model_id,
#     cache_dir="models"
# ).to("cuda")

pipe = accelerator.prepare(pipe)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from lpw_stable_diffusion_xl import get_weighted_text_embeddings_sdxl
import os
from datetime import datetime
from PIL import Image, PngImagePlugin
import gc
import random
import torch
import time 

# Create an image folder with the current date
today_date = datetime.now().strftime("%Y-%m-%d")
folder_path = os.path.join(os.getcwd(), today_date)
os.makedirs(folder_path, exist_ok=True)  # Create directory if it doesn't exist
determinism = True

num_inference_steps = 26
guidance_scale = 7.5
num_images = 1
image_width = 768
image_height = 1024

prompt = """
very long prompt
"""

negative_prompt = """
very long negative prompt
"""

prompt_embeds,negative_prompt_embeds,pooled_prompt_embeds,negative_pooled_prompt_embeds = get_weighted_text_embeddings_sdxl(pipe, prompt = prompt, neg_prompt = negative_prompt)

for i in range(20):
    print(i)

    if determinism:
        random.seed(time.time())
        seed1 = random.randint(0, 10000000000)
        seed2 = random.randint(0, 10000000000)
        random.seed(seed1)
        torch.manual_seed(seed2)

    generated_image = pipe(prompt_embeds=prompt_embeds, negative_prompt_embeds=negative_prompt_embeds,pooled_prompt_embeds=pooled_prompt_embeds, negative_pooled_prompt_embeds=negative_pooled_prompt_embeds,num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, 
                 num_images_per_prompt=num_images, height=image_height, width=image_width).images[0]

    current_time = int(datetime.now().timestamp() * 1000)  # Current time in milliseconds
    file_path = os.path.join(folder_path, f"{current_time}.png")
    
    meta = PngImagePlugin.PngInfo()
    meta.add_text("Positive Prompt", prompt)
    meta.add_text("Negative Prompt", negative_prompt)
    meta.add_text("Model ID", model_id)  
    meta.add_text("Pytorch Seed", str(seed2))  
    meta.add_text("num_inference_steps", str(num_inference_steps))  
    meta.add_text("guidance_scale", str(guidance_scale))  
    
    if determinism: 
        meta.add_text("seed1", str(seed1))  
        meta.add_text("seed2", str(seed2))  
        
    generated_image.save(file_path, "PNG", pnginfo=meta)
    generated_image.show()

    torch.cuda.empty_cache()
    gc.collect()